<a href="https://colab.research.google.com/github/MicroprocessorX069/CheXNET-/blob/master/ChestXRay_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import os
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [0]:
num_classes=2
num_epochs=1
batch_size=100
learning_rate=0.1

In [0]:
def loadTrainData(data_dir,resize=(224,224)):
  data_transforms={
      'train': transforms.Compose([
          transforms.RandomResizedCrop(max(resize)),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.485,0.486,0.406],[0.229,0.224,0.225])
      ]),
      'val': transforms.Compose([
          transforms.RandomResizedCrop(max(resize)),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.485,0.486,0.406],[0.229,0.224,0.225])
      ]),
      'test': transforms.Compose([
          transforms.RandomResizedCrop(max(resize)),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.485,0.486,0.406],[0.229,0.224,0.225])
      ])
      
  }
  dataset={x:datasets.ImageFolder(os.path.join(data_dir,x),data_transforms[x])
          for x in ['train','test','val']}
  dataset_loader={x:torch.utils.data.DataLoader(dataset[x],
                                                batch_size=5,
                                               shuffle=True)
                 for x in ['train','test','val']}
  dataset_size={x:len(dataset[x]) for x in ['train','test','val']}
  dataset_classes=dataset['train'].classes
  return dataset_loader['train'],dataset_loader['val'],dataset_loader['test']

In [0]:
data_dir="/content/drive/My Drive/Projects/Pneumonia detection/chest_xray/chest_xray/"
train_loader, val_loader,test_loader=loadTrainData(data_dir)

##Model

In [0]:
class DenseNet121(nn.Module):
  def __init__(self,out_size):
    super(DenseNet121,self).__init__()
    self.densenet121=torchvision.models.densenet121()
    num_features=self.densenet121.classifier.in_features
    self.densenet121.classifier=nn.Sequential(
    nn.Linear(num_features,out_size),
    nn.Sigmoid())
  def forward(self,x):
    x=self.densenet121(x)
    return x

model=DenseNet121(num_classes).to(device)

In [0]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)


In [45]:
total_step=len(train_loader)
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    images=images.to(device)
    labels=labels.to(device)
    
    outputs=model(images)
    loss=criterion(outputs,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if(i+1)%100==0:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
        .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/1], Step [100/1044], Loss: 0.5133
Epoch [1/1], Step [200/1044], Loss: 0.7133
Epoch [1/1], Step [300/1044], Loss: 0.5133
Epoch [1/1], Step [400/1044], Loss: 0.3133
Epoch [1/1], Step [500/1044], Loss: 0.9133
Epoch [1/1], Step [600/1044], Loss: 0.7133
Epoch [1/1], Step [700/1044], Loss: 0.5133
Epoch [1/1], Step [800/1044], Loss: 0.7133
Epoch [1/1], Step [900/1044], Loss: 0.5133
Epoch [1/1], Step [1000/1044], Loss: 0.5133
